Download pretrained model.

In [ ]:
google.colab.files.upload()

In [ ]:
!pip install --upgrade "accelerate>=0.26.0"

In [ ]:
!pip install transformers

In [ ]:
# Disable wandb to prevent interactive prompts
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-multilingual-cased"
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Loading model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unpack training data.

In [ ]:
!unzip subtask1.zip -d /content/

Load training data. (from POLAR@SEM-EVAL2026 https://polar-semeval.github.io/tasks.html)

In [3]:
import glob
import os
from datasets import load_dataset

# Check current directory
print(f"Current directory: {os.getcwd()}")

# Collect all CSV paths for train and dev
train_files = glob.glob("train/*.csv")
dev_files = glob.glob("dev/*.csv")

# Load them into a DatasetDict
dataset = load_dataset(
    "csv",
    data_files={"train": train_files, "test": dev_files},
    column_names=["id", "text", "label"]  # 'polarization' mapped to 'label' for Trainer compatibility
)

Current directory: /content


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Tokenize data.

In [4]:
import glob
import torch
from datasets import load_dataset
from transformers import AutoTokenizer

# 1. Setup Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# 2. Identify Files - ONLY use train (dev has no labels!)
train_files = glob.glob("train/*.csv")

# 3. Load CSVs with proper header handling
raw_datasets = load_dataset(
    "csv",
    data_files={"train": train_files},
    # Don't override column_names - let it read the header
)

# Rename 'polarization' to 'label' for Trainer compatibility
raw_datasets = raw_datasets.rename_column("polarization", "label")

# 4. ROBUST PREPROCESSING
def clean_and_tokenize(examples):
    clean_labels = []
    for val in examples["label"]:
        try:
            clean_labels.append(int(str(val).strip()))
        except (ValueError, TypeError):
            clean_labels.append(0)

    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    tokenized["labels"] = clean_labels
    return tokenized

# 5. Apply preprocessing
tokenized_datasets = raw_datasets.map(
    clean_and_tokenize,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

# 6. Split training data: 92% train, 8% eval for better error analysis
split_datasets = tokenized_datasets["train"].train_test_split(test_size=0.08, seed=42)
shuffled_datasets = {
    "train": split_datasets["train"].shuffle(seed=42),
    "test": split_datasets["test"]  # This is our holdout for evaluation
}

print(f"Training samples: {len(shuffled_datasets['train'])}")
print(f"Evaluation samples: {len(shuffled_datasets['test'])}")
print("Sample labels:", shuffled_datasets["train"]["labels"][:20])

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/73681 [00:00<?, ? examples/s]

Training samples: 67786
Evaluation samples: 5895
Sample labels: [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


Define evaluation metrics.

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # mBERT outputs logits; we take the argmax to get the 0 or 1 prediction
    predictions = np.argmax(logits, axis=-1)

    # 'macro' average treats both classes (0 and 1) as equally important
    # regardless of how many samples each has in your CSVs.
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro'
    )
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1_macro': f1,
        'precision_macro': precision,
        'recall_macro': recall
    }

Test baseline performance. (on **GPU**!)

In [6]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
import pandas as pd
import glob
import os
from tqdm import tqdm

# Load model with classification head
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Set up arguments for prediction
pred_args = TrainingArguments(
    output_dir="./baseline",
    per_device_eval_batch_size=32,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=pred_args,
)

# Create baseline output directory
os.makedirs("./baseline", exist_ok=True)

# Get all dev files (these have ids but no labels - for submission)
dev_files = glob.glob("dev/*.csv")

print(f"Generating baseline predictions for {len(dev_files)} languages...")

for dev_file in tqdm(dev_files):
    # Extract language code from filename (e.g., "dev/eng.csv" -> "eng")
    lang_code = os.path.basename(dev_file).replace('.csv', '')
    
    # Load dev data
    dev_df = pd.read_csv(dev_file)
    
    # Tokenize the text
    dev_encodings = tokenizer(
        dev_df['text'].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    
    # Create a simple dataset
    class SimpleDataset:
        def __init__(self, encodings):
            self.encodings = encodings
        
        def __getitem__(self, idx):
            return {key: val[idx] for key, val in self.encodings.items()}
        
        def __len__(self):
            return len(self.encodings['input_ids'])
    
    dev_dataset = SimpleDataset(dev_encodings)
    
    # Get predictions
    predictions = trainer.predict(dev_dataset)

    # DEBUG: Check the raw logits
    print(f"\nDEBUG - First 5 samples for {lang_code}:")
    print("Raw logits (class 0, class 1):")
    print(predictions.predictions[:5])
    print("\nArgmax predictions:")
    print(predictions.predictions.argmax(axis=-1)[:5])

    pred_labels = predictions.predictions.argmax(axis=-1)
    
    # Create submission dataframe
    submission_df = pd.DataFrame({
        'id': dev_df['id'],
        'polarization': pred_labels
    })
    
    # Save in submission format
    output_path = f"./baseline/pred_{lang_code}.csv"
    submission_df.to_csv(output_path, index=False)
    
    print(f"  ✓ {lang_code}: {len(submission_df)} predictions -> {output_path}")

print(f"\n✓ All baseline predictions saved to ./baseline/")
print(f"  Files: pred_{{lang_code}}.csv for each language")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Generating baseline predictions for 22 languages...


  0%|          | 0/22 [00:00<?, ?it/s]

  5%|▍         | 1/22 [00:01<00:30,  1.43s/it]


DEBUG - First 5 samples for rus:
Raw logits (class 0, class 1):
[[ 0.01011206  0.06056256]
 [-0.01397478  0.003688  ]
 [ 0.01442621  0.01861575]
 [ 0.03572325 -0.01262246]
 [ 0.04233878 -0.0022111 ]]

Argmax predictions:
[1 1 1 0 0]
  ✓ rus: 167 predictions -> ./baseline/pred_rus.csv


...


DEBUG - First 5 samples for fas:
Raw logits (class 0, class 1):
[[-0.03931858  0.05743727]
 [-0.0735813   0.05618145]
 [-0.06598835  0.14003749]
 [-0.08615829  0.12371714]
 [-0.04689865  0.1561013 ]]

Argmax predictions:
[1 1 1 1 1]
  ✓ fas: 164 predictions -> ./baseline/pred_fas.csv


 95%|█████████▌| 21/22 [00:23<00:01,  1.21s/it]


DEBUG - First 5 samples for amh:
Raw logits (class 0, class 1):
[[-0.01682358 -0.0459111 ]
 [ 0.071169   -0.06170953]
 [ 0.03825536 -0.07965112]
 [ 0.03361867 -0.06030038]
 [-0.02911301  0.02754902]]

Argmax predictions:
[0 0 0 0 1]
  ✓ amh: 166 predictions -> ./baseline/pred_amh.csv


100%|██████████| 22/22 [00:23<00:00,  1.08s/it]


DEBUG - First 5 samples for pol:
Raw logits (class 0, class 1):
[[ 0.02540866  0.04773212]
 [-0.0482153   0.12846759]
 [-0.02412865  0.0427485 ]
 [-0.02745161  0.14427033]
 [-0.06660207  0.05988966]]

Argmax predictions:
[1 1 1 1 1]
  ✓ pol: 119 predictions -> ./baseline/pred_pol.csv

✓ All baseline predictions saved to ./baseline/
  Files: pred_{lang_code}.csv for each language


Train the classification head.

In [7]:
import gc
import torch

# Delete any existing model/trainer from previous cells
for var_name in ['model', 'trainer', 'baseline_results']:
    if var_name in dir():
        exec(f"del {var_name}")

# Force garbage collection
gc.collect()

# Clear all CUDA memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    
    # Show current memory state
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"GPU Memory after cleanup:")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved:  {reserved:.2f} GB")
    print(f"  Total:     {total:.2f} GB")
    print(f"  Free:      {total - reserved:.2f} GB")

GPU Memory after cleanup:
  Allocated: 0.01 GB
  Reserved:  0.01 GB
  Total:     15.83 GB
  Free:      15.81 GB


In [8]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch
import os

# Memory optimization environment variable
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

# Load fresh model with classification head
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Training arguments - SPEED OPTIMIZED for T4 GPU with clean memory
training_args = TrainingArguments(
    output_dir="./mbert_polarization",
    
    # Training settings - MAXIMIZED for speed
    num_train_epochs=3,
    per_device_train_batch_size=16,   # Increased from 4
    per_device_eval_batch_size=32,    # Increased from 8
    gradient_accumulation_steps=2,    # Reduced: effective batch = 16 * 2 = 32
    
    # Learning rate schedule
    learning_rate=2e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # Evaluation & saving - less frequent = faster
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    
    # Logging - less frequent
    logging_steps=200,
    report_to="none",
    
    # T4 SPEED OPTIMIZATIONS
    fp16=True,                        # Tensor Cores = 2-3x faster
    bf16=False,                       # T4 doesn't support bf16
    dataloader_pin_memory=True,       # Faster CPU->GPU transfer
    dataloader_num_workers=4,         # More parallel data loading
    optim="adamw_torch_fused",        # Fused optimizer (faster)
    torch_compile=False,              # Skip compilation overhead
    
    # No gradient checkpointing = faster (uses more memory but we have it now)
    gradient_checkpointing=False,
    max_grad_norm=1.0,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=shuffled_datasets["train"],
    eval_dataset=shuffled_datasets["test"],
    compute_metrics=compute_metrics,
)

# Train!
print("Starting training (T4 GPU - SPEED optimized)...")
print(f"Using device: {trainer.args.device}")
print(f"Batch size: {training_args.per_device_train_batch_size} x {training_args.gradient_accumulation_steps} = {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
trainer.train()

# Final evaluation
final_results = trainer.evaluate()
print(f"\n=== Training Complete ===")
print(f"Final Macro F1: {final_results['eval_f1_macro']:.4f}")
print(f"Final Accuracy: {final_results['eval_accuracy']:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training (T4 GPU - SPEED optimized)...
Using device: cuda:0
Batch size: 16 x 2 = 32


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,0.486400,0.448661,0.787108,0.786337,0.786541,0.786178
2,0.386800,0.471649,0.789483,0.789481,0.792192,0.792009
3,0.292800,0.516089,0.798134,0.797706,0.797527,0.798011


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: T


=== Training Complete ===
Final Macro F1: 0.7977
Final Accuracy: 0.7981


Analyze predictions on 8% holdout set

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import os

# Create analysis directory
os.makedirs("./analysis", exist_ok=True)

# Get predictions on the 8% holdout set
print("Generating predictions on 8% holdout set...")
test_predictions = trainer.predict(shuffled_datasets["test"])
pred_labels = test_predictions.predictions.argmax(axis=-1)
true_labels = np.array(shuffled_datasets["test"]["labels"])

# Calculate metrics
print("\n=== Evaluation Metrics ===")
print(f"Accuracy: {(pred_labels == true_labels).mean():.4f}")
print(f"\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=['Non-polarized (0)', 'Polarized (1)']))

print(f"\nConfusion Matrix:")
cm = confusion_matrix(true_labels, pred_labels)
print(f"                  Predicted 0  Predicted 1")
print(f"Actual 0 (TN/FP):      {cm[0,0]:5d}        {cm[0,1]:5d}")
print(f"Actual 1 (FN/TP):      {cm[1,0]:5d}        {cm[1,1]:5d}")

# Need to reconstruct the original text from the test set
# We'll need to reload the data to get the text and IDs
print("\n=== Creating detailed analysis CSV ===")

# Load all training data again to match with indices
train_files = glob.glob("train/*.csv")
all_data = []
for file in train_files:
    df = pd.read_csv(file)
    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Split the same way to get the test indices
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(combined_df, test_size=0.08, random_state=42)

# Create analysis dataframe
analysis_df = pd.DataFrame({
    'id': test_df['id'].values,
    'text': test_df['text'].values,
    'true_label': true_labels,
    'predicted_label': pred_labels,
    'correct': pred_labels == true_labels
})

# Add language code (extracted from id)
analysis_df['language'] = analysis_df['id'].str.split('_').str[0]

# Sort by incorrect predictions first, then by language
analysis_df = analysis_df.sort_values(['correct', 'language', 'id'])

# Save overall analysis
output_file = './analysis/predictions_analysis.csv'
analysis_df.to_csv(output_file, index=False)
print(f"✓ Saved overall analysis: {output_file}")
print(f"  Total samples: {len(analysis_df)}")
print(f"  Correct: {analysis_df['correct'].sum()} ({100*analysis_df['correct'].mean():.2f}%)")
print(f"  Incorrect: {(~analysis_df['correct']).sum()} ({100*(~analysis_df['correct']).mean():.2f}%)")

# Save errors only (for easier manual review)
errors_df = analysis_df[~analysis_df['correct']]
errors_file = './analysis/errors_only.csv'
errors_df.to_csv(errors_file, index=False)
print(f"\n✓ Saved errors only: {errors_file}")
print(f"  Total errors: {len(errors_df)}")

# Save per-language breakdown
print(f"\n=== Per-Language Performance ===")
lang_stats = analysis_df.groupby('language').agg({
    'correct': ['count', 'sum', 'mean']
}).round(4)
lang_stats.columns = ['total', 'correct', 'accuracy']
lang_stats = lang_stats.sort_values('accuracy')

lang_stats_file = './analysis/per_language_accuracy.csv'
lang_stats.to_csv(lang_stats_file)
print(lang_stats)
print(f"\n✓ Saved language stats: {lang_stats_file}")

# Save False Positives and False Negatives separately
fp_df = analysis_df[(analysis_df['true_label'] == 0) & (analysis_df['predicted_label'] == 1)]
fn_df = analysis_df[(analysis_df['true_label'] == 1) & (analysis_df['predicted_label'] == 0)]

if len(fp_df) > 0:
    fp_file = './analysis/false_positives.csv'
    fp_df.to_csv(fp_file, index=False)
    print(f"\n✓ False Positives: {len(fp_df)} samples → {fp_file}")

if len(fn_df) > 0:
    fn_file = './analysis/false_negatives.csv'
    fn_df.to_csv(fn_file, index=False)
    print(f"✓ False Negatives: {len(fn_df)} samples → {fn_file}")

print(f"\n{'='*60}")
print(f"All analysis files saved to ./analysis/")
print(f"{'='*60}")

Generating predictions on 8% holdout set...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



=== Evaluation Metrics ===
Accuracy: 0.7981

Classification Report:
                   precision    recall  f1-score   support

Non-polarized (0)       0.78      0.80      0.79      2786
    Polarized (1)       0.81      0.80      0.81      3109

         accuracy                           0.80      5895
        macro avg       0.80      0.80      0.80      5895
     weighted avg       0.80      0.80      0.80      5895


Confusion Matrix:
                  Predicted 0  Predicted 1
Actual 0 (TN/FP):       2217          569
Actual 1 (FN/TP):        621         2488

=== Creating detailed analysis CSV ===
✓ Saved overall analysis: ./analysis/predictions_analysis.csv
  Total samples: 5895
  Correct: 4705 (79.81%)
  Incorrect: 1190 (20.19%)

✓ Saved errors only: ./analysis/errors_only.csv
  Total errors: 1190

=== Per-Language Performance ===
          total  correct  accuracy
language                          
ben         272      207    0.7610
deu         259      198    0.7645
nep     

In [11]:
import pandas as pd
import glob
import os
from tqdm import tqdm

# Create output directory for trained model predictions
os.makedirs("./trained_predictions", exist_ok=True)

# Get all dev files (these have ids but no labels - for submission)
dev_files = glob.glob("dev/*.csv")

print(f"Generating trained model predictions for {len(dev_files)} languages...")

for dev_file in tqdm(dev_files):
    # Extract language code from filename (e.g., "dev/eng.csv" -> "eng")
    lang_code = os.path.basename(dev_file).replace('.csv', '')
    
    # Load dev data
    dev_df = pd.read_csv(dev_file)
    
    # Tokenize the text
    dev_encodings = tokenizer(
        dev_df['text'].tolist(),
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )
    
    # Create a simple dataset
    class SimpleDataset:
        def __init__(self, encodings):
            self.encodings = encodings
        
        def __getitem__(self, idx):
            return {key: val[idx] for key, val in self.encodings.items()}
        
        def __len__(self):
            return len(self.encodings['input_ids'])
    
    dev_dataset = SimpleDataset(dev_encodings)
    
    # Get predictions from TRAINED model using existing trainer
    predictions = trainer.predict(dev_dataset)
    pred_labels = predictions.predictions.argmax(axis=-1)
    
    # Create submission dataframe
    submission_df = pd.DataFrame({
        'id': dev_df['id'],
        'polarization': pred_labels
    })
    
    # Save in submission format
    output_path = f"./trained_predictions/pred_{lang_code}.csv"
    submission_df.to_csv(output_path, index=False)
    
    print(f"  ✓ {lang_code}: {len(submission_df)} predictions -> {output_path}")

print(f"\n✓ All trained model predictions saved to ./trained_predictions/")
print(f"  Files: pred_{{lang_code}}.csv for each language")
print(f"\nYou can now submit the files from ./trained_predictions/ to Codabench!")

Generating trained model predictions for 22 languages...


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Generating trained model predictions for 22 languages...


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Generating trained model predictions for 22 languages...


...

 95%|█████████▌| 21/22 [00:11<00:00,  1.68it/s]

  ✓ amh: 166 predictions -> ./trained_predictions/pred_amh.csv


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


100%|██████████| 22/22 [00:12<00:00,  1.78it/s]

  ✓ pol: 119 predictions -> ./trained_predictions/pred_pol.csv

✓ All trained model predictions saved to ./trained_predictions/
  Files: pred_{lang_code}.csv for each language

You can now submit the files from ./trained_predictions/ to Codabench!


## Grid Search with Checkpointing

Hyperparameters to search:
- **Softmax offset (threshold)**: 0.4, 0.5, 0.6
- **Output dimension**: 2 buckets, 3→2→softmax, 4→2→softmax

Results are saved incrementally so training can resume if interrupted.

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
import numpy as np
import glob
import os
import json
from itertools import product
from tqdm import tqdm

# ============================================================================
# CUSTOM MODEL WITH CONFIGURABLE OUTPUT DIMENSIONS
# ============================================================================

class CustomClassificationHead(nn.Module):
    """Classification head with configurable intermediate dimensions."""
    def __init__(self, hidden_size, output_dim, num_labels=2):
        super().__init__()
        self.output_dim = output_dim
        
        if output_dim == 2:
            # Standard: hidden -> 2 -> softmax
            self.classifier = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(hidden_size, num_labels)
            )
        elif output_dim == 3:
            # 3 -> 2: hidden -> 3 -> 2 -> softmax
            self.classifier = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(hidden_size, 3),
                nn.ReLU(),
                nn.Linear(3, num_labels)
            )
        elif output_dim == 4:
            # 4 -> 2: hidden -> 4 -> 2 -> softmax
            self.classifier = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(hidden_size, 4),
                nn.ReLU(),
                nn.Linear(4, num_labels)
            )
    
    def forward(self, x):
        return self.classifier(x)


class CustomMBertForClassification(nn.Module):
    """mBERT with custom classification head."""
    def __init__(self, model_name, output_dim, num_labels=2):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.classifier = CustomClassificationHead(
            self.bert.config.hidden_size, 
            output_dim, 
            num_labels
        )
        self.num_labels = num_labels
        self.output_dim = output_dim
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        return {"loss": loss, "logits": logits}


# ============================================================================
# GRID SEARCH CONFIGURATION
# ============================================================================

GRID_SEARCH_PARAMS = {
    'threshold': [0.4, 0.5, 0.6],
    'output_dim': [2, 3, 4]
}

# Generate all combinations
param_combinations = list(product(
    GRID_SEARCH_PARAMS['threshold'],
    GRID_SEARCH_PARAMS['output_dim']
))

print(f"Grid Search: {len(param_combinations)} combinations")
for i, (thresh, out_dim) in enumerate(param_combinations):
    print(f"  {i+1}. threshold={thresh}, output_dim={out_dim}")


# ============================================================================
# CHECKPOINT MANAGEMENT
# ============================================================================

GRID_SEARCH_DIR = "./grid_search_results"
CHECKPOINT_FILE = os.path.join(GRID_SEARCH_DIR, "checkpoint.json")

os.makedirs(GRID_SEARCH_DIR, exist_ok=True)

def load_checkpoint():
    """Load checkpoint to resume from last completed combination."""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return json.load(f)
    return {"completed": [], "results": {}}

def save_checkpoint(checkpoint):
    """Save checkpoint after each completed combination."""
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint, f, indent=2)

def get_combo_key(threshold, output_dim):
    """Create unique key for each parameter combination."""
    return f"thresh_{threshold}_outdim_{output_dim}"

checkpoint = load_checkpoint()
print(f"\nCheckpoint loaded: {len(checkpoint['completed'])} combinations already completed")

In [ ]:
# ============================================================================
# RUN GRID SEARCH WITH CHECKPOINTING
# ============================================================================

import gc

def apply_threshold(logits, threshold=0.5):
    """Apply custom threshold to convert logits to predictions.
    
    Standard softmax uses 0.5 threshold (argmax).
    threshold < 0.5 = more likely to predict class 1 (polarized)
    threshold > 0.5 = more likely to predict class 0 (non-polarized)
    """
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    # Class 1 probability vs threshold
    predictions = (probs[:, 1] >= threshold).astype(int)
    return predictions


def train_and_evaluate(output_dim, threshold, combo_key):
    """Train model with given output_dim and evaluate with given threshold."""
    
    print(f"\n{'='*60}")
    print(f"Training: output_dim={output_dim}, threshold={threshold}")
    print(f"{'='*60}")
    
    # Clear memory
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Create custom model
    model = CustomMBertForClassification("bert-base-multilingual-cased", output_dim=output_dim)
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./grid_search_results/{combo_key}",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=2,
        learning_rate=2e-5,
        warmup_ratio=0.1,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        logging_steps=200,
        report_to="none",
        fp16=True if torch.cuda.is_available() else False,
        dataloader_pin_memory=True,
        dataloader_num_workers=4,
    )
    
    # Custom compute_metrics that uses the threshold
    def compute_metrics_with_threshold(eval_pred):
        logits, labels = eval_pred
        predictions = apply_threshold(logits, threshold)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions, average='macro'
        )
        acc = accuracy_score(labels, predictions)
        
        return {
            'accuracy': acc,
            'f1_macro': f1,
            'precision_macro': precision,
            'recall_macro': recall
        }
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=shuffled_datasets["train"],
        eval_dataset=shuffled_datasets["test"],
        compute_metrics=compute_metrics_with_threshold,
    )
    
    # Train
    trainer.train()
    
    # Evaluate on holdout
    eval_results = trainer.evaluate()
    print(f"Eval F1 Macro: {eval_results['eval_f1_macro']:.4f}")
    print(f"Eval Accuracy: {eval_results['eval_accuracy']:.4f}")
    
    return trainer, eval_results


def generate_predictions_for_combo(trainer, threshold, combo_key):
    """Generate predictions for all dev files in Codabench format."""
    
    output_dir = f"./grid_search_results/{combo_key}/predictions"
    os.makedirs(output_dir, exist_ok=True)
    
    dev_files = glob.glob("dev/*.csv")
    
    print(f"\nGenerating predictions for {len(dev_files)} languages...")
    
    for dev_file in tqdm(dev_files, desc="Predicting"):
        lang_code = os.path.basename(dev_file).replace('.csv', '')
        dev_df = pd.read_csv(dev_file)
        
        # Tokenize
        dev_encodings = tokenizer(
            dev_df['text'].tolist(),
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors="pt"
        )
        
        # Simple dataset
        class SimpleDataset:
            def __init__(self, encodings):
                self.encodings = encodings
            def __getitem__(self, idx):
                return {key: val[idx] for key, val in self.encodings.items()}
            def __len__(self):
                return len(self.encodings['input_ids'])
        
        dev_dataset = SimpleDataset(dev_encodings)
        
        # Get predictions
        predictions = trainer.predict(dev_dataset)
        
        # Apply custom threshold
        pred_labels = apply_threshold(predictions.predictions, threshold)
        
        # Create submission dataframe (CODABENCH FORMAT)
        submission_df = pd.DataFrame({
            'id': dev_df['id'],
            'polarization': pred_labels
        })
        
        # Save in submission format
        output_path = f"{output_dir}/pred_{lang_code}.csv"
        submission_df.to_csv(output_path, index=False)
    
    print(f"✓ Predictions saved to {output_dir}/")
    return output_dir


# ============================================================================
# MAIN GRID SEARCH LOOP
# ============================================================================

print("\n" + "="*60)
print("STARTING GRID SEARCH")
print("="*60)

for threshold, output_dim in param_combinations:
    combo_key = get_combo_key(threshold, output_dim)
    
    # Skip if already completed
    if combo_key in checkpoint["completed"]:
        print(f"\n⏭️  Skipping {combo_key} (already completed)")
        continue
    
    try:
        # Train and evaluate
        trainer, eval_results = train_and_evaluate(output_dim, threshold, combo_key)
        
        # Generate predictions in Codabench format
        pred_dir = generate_predictions_for_combo(trainer, threshold, combo_key)
        
        # Save results to checkpoint
        checkpoint["results"][combo_key] = {
            "threshold": threshold,
            "output_dim": output_dim,
            "eval_f1_macro": eval_results['eval_f1_macro'],
            "eval_accuracy": eval_results['eval_accuracy'],
            "predictions_dir": pred_dir
        }
        checkpoint["completed"].append(combo_key)
        save_checkpoint(checkpoint)
        
        print(f"\n✅ Completed {combo_key}")
        print(f"   F1 Macro: {eval_results['eval_f1_macro']:.4f}")
        
        # Clean up memory
        del trainer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    except Exception as e:
        print(f"\n❌ Error in {combo_key}: {str(e)}")
        print("   Checkpoint saved. You can resume from here.")
        save_checkpoint(checkpoint)
        raise e

print("\n" + "="*60)
print("GRID SEARCH COMPLETE!")
print("="*60)

In [ ]:
# ============================================================================
# SHOW GRID SEARCH RESULTS SUMMARY
# ============================================================================

print("\n" + "="*60)
print("GRID SEARCH RESULTS SUMMARY")
print("="*60)

# Load final checkpoint
final_checkpoint = load_checkpoint()

# Create results dataframe
results_list = []
for combo_key, result in final_checkpoint["results"].items():
    results_list.append({
        "threshold": result["threshold"],
        "output_dim": result["output_dim"],
        "f1_macro": result["eval_f1_macro"],
        "accuracy": result["eval_accuracy"],
        "predictions_dir": result["predictions_dir"]
    })

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values("f1_macro", ascending=False)

print("\nResults sorted by F1 Macro:")
print(results_df.to_string(index=False))

# Find best combination
best_result = results_df.iloc[0]
print(f"\n🏆 BEST COMBINATION:")
print(f"   Threshold: {best_result['threshold']}")
print(f"   Output Dim: {best_result['output_dim']}")
print(f"   F1 Macro: {best_result['f1_macro']:.4f}")
print(f"   Accuracy: {best_result['accuracy']:.4f}")
print(f"\n   Predictions ready for Codabench at:")
print(f"   {best_result['predictions_dir']}/")

# Save results summary
results_df.to_csv("./grid_search_results/results_summary.csv", index=False)
print(f"\n✓ Results summary saved to ./grid_search_results/results_summary.csv")